In [1]:
import os, json
from collections import Counter 
with open('KES.json', 'r') as fp:
    heroes = json.load(fp)
    print(heroes[17])

{'localized_name': 'Monkey King', 'name': 'npc_dota_hero_monkey_king', 'id': 18, 'cat_wins': {'L1': 48.27, 'L2': 49.46, 'L3': 49.95, 'L4': 50.18, 'L5': 49.71}, 'data': {'best_vs': ['Meepo', 'Broodmother', 'Alchemist', 'Underlord', 'Anti-Mage'], 'best_vs_advantage': [6.64, 5.97, 5.4, 2.93, 2.83], 'worst_vs': ['Tinker', 'Outworld Devourer', 'Storm Spirit', 'Viper', 'Queen of Pain'], 'worst_vs_disadvantage': [3.37, 3.31, 3.03, 3, 2.71]}}


In [2]:
import os
import json
from collections import Counter

with open('KES.json', 'r') as fp:
    heroes = json.load(fp)


def get_hero_human_readable(hero_id):
    for hero in heroes:
        if hero['id'] == hero_id:
            return hero['localized_name']
    return 'Unknown hero: %d' % hero_id


class Engine:
    def __init__(self, algorithm):
        self.algorithm = algorithm

    def recommend(self,my_team, their_team,medal_lvl):
        '''Returns a list of (hero, probablility of winning with hero added) recommended to complete my_team.'''
        my_team = list(my_team)
        their_team = list(their_team)
        
        score = 0.0
        medal_val = 0.0

        medal_team = 0.0
        medal_enemy = 0.0
        
        current_hero_names = []
        
        selection = []
        
        score_absolute  = 10

        for hero_id_team in my_team:
            
            hero_data_team = heroes[hero_id_team - 1]
            
            current_hero_names.append(hero_data_team['localized_name'])
            
            lvl_id = 'L'+str(medal_lvl)
            medal_team += hero_data_team['cat_wins'][lvl_id]

            for hero_id_enemy in their_team:
                hero_data_enemy = heroes[hero_id_enemy - 1]
                
                if hero_data_enemy['localized_name'] not in current_hero_names:
                    current_hero_names.append(hero_data_enemy['localized_name'])
                    
                if hero_data_team['localized_name'] in hero_data_enemy['data']['best_vs']:
                    index_of_hero = hero_data_enemy['data']['best_vs'].index(hero_data_team['localized_name'])
                    score = score - hero_data_enemy['data']['best_vs_advantage'][index_of_hero]
    
                if hero_data_team['localized_name'] in hero_data_enemy['data']['worst_vs']:
                    index_of_hero = hero_data_enemy['data']['worst_vs'].index(hero_data_team['localized_name'])
                    score = score + hero_data_enemy['data']['worst_vs_disadvantage'][index_of_hero]
                    
                
        for hero_id_enemy in their_team:
            
            hero_data_enemy = heroes[hero_id_enemy - 1]
            
            lvl_id = 'L'+str(medal_lvl)
            medal_enemy += hero_data_enemy['cat_wins'][lvl_id]
            

            for i in range(len(hero_data_enemy['data']['worst_vs'])):
                if hero_data_enemy['data']['worst_vs'][i] not in current_hero_names:
                    selection.append((hero_data_enemy['data']['worst_vs'][i],int(hero_data_enemy['data']['worst_vs_disadvantage'][i]*100)))
            
            # printing original list 
        #print("The original list is : " + str(selection) )

        # Aggregate values by tuple keys 
        # using Counter() + generator expression 
        res = list(Counter(key for key, num in selection  
                      for idx in range(num)).items()) 
        
        values_only = [y for x,y in res]
        if len(values_only) > 0:
            score_absolute += max(values_only)
        
        selection = [(round(y / (score_absolute), 2),next((name['id'] for name in heroes if name['localized_name'] == x), None)) for x,y in res]
        
        selection = sorted(selection, key=lambda x: x[0], reverse=True)

        # printing result 
        
        if len(my_team) > 0 and len(their_team) > 0:
            medal_team = medal_team / len(my_team)
            medal_enemy = medal_enemy / len(their_team)
        
            medal_val = medal_team * 100 / (medal_team + medal_enemy)
        
            medal_val = ( medal_val + score ) / 100
        else:
            medal_val = 0.0

        
        if len(selection) > 5:
            selection = selection[:5]
        
        return selection,medal_val

In [3]:
from flask import Flask, render_template, request
# from engine import Engine
import json

URL_PREFIX = ''

app = Flask(__name__)
#engine = Engine(D2KNearestNeighbors())
engine = Engine("")

def get_api_string(recommendations, prob):
    recommendations = list(map(str, recommendations))
    return json.dumps({'x': recommendations, 'prob_x': prob})

@app.route("/")
def index():
    return render_template('index.html')

@app.route(URL_PREFIX + "/api/suggest/")
def api():
    if 'x' not in request.args or 'y' not in request.args or 'z' not in request.args:
        return 'Invalid request'
    my_team = request.args['x'].split(',')
    if len(my_team) == 1 and my_team[0] == '':
        my_team = []
    else:
        my_team = map(int, my_team)

    their_team = request.args['y'].split(',')
    if len(their_team) == 1 and their_team[0] == '':
        their_team = []
    else:
        their_team = map(int, their_team)

    medal_lvl = request.args['z'].split(',')
    if len(medal_lvl) == 1 and medal_lvl[0] == '':
        medal_lvl = 1
    else:
        medal_lvl = medal_lvl[0]

    prob_recommendation_pairs,prob = engine.recommend(my_team, their_team,medal_lvl)
    recommendations = [hero for prob, hero in prob_recommendation_pairs]
    print(recommendations, round(prob, 2))
    return get_api_string(recommendations, round(prob, 2))

if __name__ == "__main__":
    app.debug = False
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Apr/2020 15:21:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2020 15:21:34] "GET /static/css/textboxlist.css HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2020 15:21:34] "GET /static/css/cloud.css HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2020 15:21:34] "GET /static/css/global.css HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2020 15:21:34] "GET /static/lib/prototype.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2020 15:21:34] "GET /static/lib/TextBoxList.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2020 15:21:34] "GET /static/index.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2020 15:21:34] "GET /static/lib/S2.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2020 15:21:34] "GET /static/src/ScaleColor.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2020 15:21:34] "GET /static/src/ProtoCloud.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2020 15:21:34] "GET /static/data/levels.json HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2020 15:21:34] "GET /static/data/KES.json

[] 0.0


127.0.0.1 - - [16/Apr/2020 15:21:36] "GET /api/suggest/?x=&y=&z=2 HTTP/1.1" 200 -
127.0.0.1 - - [16/Apr/2020 15:21:36] "GET /static/lib/close.gif HTTP/1.1" 200 -


[] 0.0


...

[8, 31, 84, 42] 0.64


127.0.0.1 - - [16/Apr/2020 15:22:10] "GET /api/suggest/?x=&y=17&z=2 HTTP/1.1" 200 -


[120, 8, 31, 84, 42] 0.0


127.0.0.1 - - [16/Apr/2020 15:22:11] "GET /api/suggest/?x=&y=&z=2 HTTP/1.1" 200 -


[] 0.0
